In [1]:
import pandas as pd
import numpy as np

In [2]:
# read PA testing file
df_in = pd.read_excel('./2016-17_PSSA_Keystone_All_Data/2017 PSSA Keystone Acct (School_S).xlsx',
                      sheet_name="All Students",
                      skiprows=6,
                      header=0)

In [3]:
df_in.head()

,Unnamed: 0,Test Name,Subject,School Code,School Name,Learning Network,Grade,Category,Number Tested,Count #,Percent %,Count #.1,Percent %.1,Count #.2,Percent %.2,Count #.3,Percent %.3,Count #.4,Percent %.4
0,NaN,Keystone,Algebra 1,609,A. Philip Randolph Career and Technical High S...,NEIGHBORHOOD NETWORK 4,11,All Students,125,54,0.432,58,0.464,12,0.096,1,0.008,13,0.104
1,NaN,Keystone,Biology,609,A. Philip Randolph Career and Technical High S...,NEIGHBORHOOD NETWORK 4,11,All Students,116,95,0.818965,18,0.155172,3,0.0258621,0,0,3,0.0258621
2,NaN,Keystone,Literature,609,A. Philip Randolph Career and Technical High S...,NEIGHBORHOOD NETWORK 4,11,All Students,124,35,0.282258,58,0.467742,31,0.25,0,0,31,0.25
3,NaN,PSSA,ELA,839,A.L. Fitzpatrick School,NEIGHBORHOOD NETWORK 8,6,All Students,76,4,0.0526316,31,0.407895,29,0.381579,12,0.157895,41,0.539474
4,NaN,PSSA,ELA,839,A.L. Fitzpatrick School,NEIGHBORHOOD NETWORK 8,Grades 3-8,All Students,443,74,0.167043,166,0.374718,163,0.367946,40,0.0902935,203,0.458239


In [4]:
# column 'School Code' has a newline character in the name, replace with a space
names = list(df_in.columns.values)
names[3] = 'School ID'
df_in.columns = names
df_in['School ID'] = df_in['School ID'].astype(str)

In [5]:
# look for best grades to sample -- 11th, 8th and 4th
for g in df_in['Grade'].unique():
    try:
        gi = int(g)
    except:
        continue
    gdf = df_in[df_in['Grade']==gi]
    print('Grade ' + str(gi) + ' testing has ' + str(len(gdf['School Name'].unique())) + ' participating schools.')
    subs = gdf['Subject'].unique()
    for s in subs:
        print('    ' + str(s) + ': ' + str(sum(gdf['Subject']==s)))

Grade 11 testing has 53 participating schools.
    Algebra 1: 53
    Biology: 53
    Literature: 53
    ELA: 22
    Math: 22
    Science: 17
Grade 6 testing has 125 participating schools.
    ELA: 125
    Math: 125
Grade 8 testing has 117 participating schools.
    ELA: 116
    Math: 117
    Science: 115
Grade 4 testing has 148 participating schools.
    ELA: 148
    Math: 148
    Science: 148
Grade 7 testing has 119 participating schools.
    ELA: 119
    Math: 119
Grade 5 testing has 146 participating schools.
    ELA: 146
    Math: 146
Grade 3 testing has 147 participating schools.
    ELA: 147
    Math: 147


In [6]:
df_out = pd.DataFrame({'School ID': df_in['School ID'].unique()})

# sample some of the test scores from df_in
for g in df_in['Grade'].unique():
    try:
        gi = int(g)
    except:
        continue
    
    # select the scores to sample
    if(gi==11):
        sample = True
        subjects = ['Literature','Algebra 1','Biology',]
        csv_name = 'hs'
    elif(gi==8):
        sample = True
        subjects = ['ELA','Math','Science']
        csv_name = 'ms'
    elif(gi==4):
        sample = True
        subjects = ['ELA','Math','Science']
        csv_name = 'es'
    else:
        sample = False
    
    # sample the selected scores
    subjects_str = ['English','Math','Science'] #assumption that array subjects maps to subjects_str
    if(sample):
        gdf = df_in[df_in['Grade']==gi]
        for (i,s) in enumerate(subjects):
            sdf = gdf[gdf['Subject']==s]
            sdf = sdf[['School ID','Percent %','Percent %.1','Percent %.2','Percent %.3',]]
            sdf[csv_name.upper() + ' ' + subjects_str[i]] = 1*sdf['Percent %'] + 2*sdf['Percent %.1'] + \
                                            3*sdf['Percent %.2'] + 4*sdf['Percent %.3']
            # multiply by negative 1 for divergent bar chart
            # sdf['Percent %'] = -1*sdf['Percent %']
            # sdf['Percent %.1'] = -1*sdf['Percent %.1']
            names = ['School ID',
                     subjects_str[i]+'_'+str(gi)+'_Below',
                     subjects_str[i]+'_'+str(gi)+'_Basic',
                     subjects_str[i]+'_'+str(gi)+'_Proficient',
                     subjects_str[i]+'_'+str(gi)+'_Advanced',
                     csv_name.upper() + ' ' + subjects_str[i]]
            sdf.columns = names
            df_out = pd.merge(df_out, sdf, how='left', left_on='School ID', right_on='School ID')
        # save a df for each type of school

In [7]:
# df_out = df_out[['School ID', 'HS English', 'HS Math', 'HS Science','MS English', 'MS Math', 'MS Science', \
#                  'ES English', 'ES Math', 'ES Science']]

In [8]:
df_out.head()

,School ID,English_11_Below,English_11_Basic,English_11_Proficient,English_11_Advanced,HS English,Math_11_Below,Math_11_Basic,Math_11_Proficient,Math_11_Advanced,...,Math_4_Below,Math_4_Basic,Math_4_Proficient,Math_4_Advanced,ES Math,Science_4_Below,Science_4_Basic,Science_4_Proficient,Science_4_Advanced,ES Science
0,609,0.282258,0.467742,0.25,0,1.96774,0.432,0.464,0.096,0.008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.41791,0.373134,0.164179,0.0447761,1.83582,0.132353,0.294118,0.470588,0.102941,2.54412
2,801,0.32636,0.380753,0.276151,0.0167364,1.98326,0.309524,0.551587,0.130952,0.00793651,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.25,0.291667,0.1875,0.270833,2.47917,0.0416667,0.229167,0.416667,0.3125,3
4,262,0.00847458,0.0677966,0.889831,0.0338983,2.94915,0.0042735,0.179487,0.666667,0.149573,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_out.to_csv('./philly_test_scores.csv', sep='|', index=False)